In [1]:
import os
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using Theano backend.
Using gpu device 0: GeForce GTX 1060 6GB (CNMeM is disabled, cuDNN 5105)
/usr/local/lib/python2.7/dist-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
ls data/cnn_weights/

bottleneck_features_train.npy       fc_model.h5
bottleneck_features_validation.npy  vgg16_weights.h5


In [3]:
# path to the model weights files.
weights_path = 'data/cnn_weights/vgg16_weights.h5'
top_model_weights_path = 'data/cnn_weights/fc_model.h5'
# dimensions of our images.
img_width, img_height = 100, 100

nb_epoch = 50

In [4]:
# build the VGG16 network
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))

model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

In [5]:
# load the weights of the VGG16 networks
# (trained on ImageNet, won the ILSVRC competition in 2014)
# note: when there is a complete match between your model definition
# and your weight savefile, you can simply call model.load_weights(filename)
assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
f = h5py.File(weights_path)
for k in range(f.attrs['nb_layers']):
    if k >= len(model.layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')

Model loaded.


In [6]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

In [7]:
# add the model on top of the convolutional base
model.add(top_model)

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='mean_squared_error',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['mean_squared_error'])

In [8]:
# load our faces
import os
import matplotlib.pyplot as plt
from scipy import misc
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from spearman import score_function
import time
from scipy.ndimage import imread
%matplotlib inline
from PIL import Image

all_faces = {int(number[::-1][4:][::-1]): "data/extracted_faces/" + number
                  for number in os.listdir("data/extracted_faces")
                  if number[::-1][:4] == 'gpj.'}

bw = np.array(Image.open(all_faces[1])).astype(float)
bw = bw.mean(axis=2)/255.  # black and white normalized
bw = bw.reshape((-1,))  # 10k features

X = np.zeros((len(all_faces), 10**4))

for i in range(13000):
    if i % 1000==0:
        print i
    im = np.array(Image.open(all_faces[i+1])).astype(float)
    if len(im.shape)>2:
        X[i,:] =  im.mean(axis=2).reshape((-1,))/255.
    else:
        X[i,:] = im.reshape((-1,)) / 255.


# read .csv using pandas
y = pd.read_csv("data/training_outputs.csv", delimiter=";").set_index("ID")
y = y.as_matrix()

# store id's
X_ids = range(1, 13001)

# cut into test, train
X_train, X_test, y_train, y_test = train_test_split(X[0:10000], y, test_size=0.3, random_state=42)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [9]:
# add fake RGB channels (otherwise the architecture doesn't work :/)
X_train = X_train.repeat(3, 1).reshape((-1, 3, 100, 100))
X_test = X_test.repeat(3, 1).reshape((-1, 3, 100, 100))

In [19]:
# fine-tune the model
model.fit(X_train, y_train.reshape((-1,)),
          nb_epoch=10, batch_size=32,
          validation_data=(X_test, y_test.reshape((-1,))))

Train on 7000 samples, validate on 3000 samples
Epoch 1/10
7000/7000 [==============================] - 33s - loss: 18.6051 - mean_squared_error: 18.6051 - val_loss: 14.8825 - val_mean_squared_error: 14.8825  64/7000 [..............................] - ETA: 29s - loss: 13.9348 - mean_squared_error: 13.9348

In [20]:
from spearman import score_function
preds_test = model.predict(X_test)

In [21]:
score_function(y_pred=np.round(preds_test), y_true=y_test)

0.43964441013818534